## A MultiServer Queue

Let's develop a simulation for single FIFO Queue that leads to 3 servers.  
 - Customer arrivals follow a poisson process with rate $\lambda$, (i.e. the interarrival times are exponentially distributed with mean $1/\lambda$).
 - Service Times are follow an exponential distribution with mean $1/\mu$.
 - There are $c$ different servers.

 We want to simulate one replication path with $numCustomer$ arrivals.

In [1]:
import numpy as np  

Write a function that takes in $\lambda$, $\mu$, $c$, numCustomers, simulates one replication path, and returns ther arrays A, S, D of the arrival times, start of service, and departure times for the first numCustomer arrivals.

In [13]:
def sim_one_path(lam, mu, c, numCustomers):
    A = np.zeros(numCustomers) # Arrival Times
    S = np.zeros(numCustomers) # Service Start Times
    D = np.zeros(numCustomers) # Departure Times

    server_completes_at = np.full(c, np.inf) # serverState[i] is the time when server i is free
    customers_in_service = [-1] * c
    queue = [] # index of customers that are currently waiting
    indx_arriving_customer = 0
    next_arrival = np.random.exponential(1/lam) # time of next arrival
    A[0] = next_arrival
    now = 0
    numCompletedService = 0

    while True:
        next_completion = np.min(server_completes_at)
        # next event is an arrival
        if next_arrival < next_completion: # someone arrives
            now = next_arrival
            # check if there is a free server
            if np.max(server_completes_at) == np.inf:
                indx_free_server = np.argmax(server_completes_at) # find the first free server
                
                # assign customer to free server
                customers_in_service[indx_free_server] = indx_arriving_customer
                server_completes_at[indx_free_server] = now + np.random.exponential(1/mu)
                S[indx_arriving_customer] = now
                # update for next arriving customer
                indx_arriving_customer += 1
                A[indx_arriving_customer] = now + np.random.exponential(1/lam)
            else: # no free server, so add to the queue
                queue.append(indx_arriving_customer)
                # update for next arriving customer
                
            indx_arriving_customer += 1
            A[indx_arriving_customer] = now + np.random.exponential(1/lam)
        else: # next event is a completion
            now = next_completion
            
            #find the server that completes first:
            indx_completing_server = np.argmin(server_completes_at)
            indx_leaving_customer = customers_in_service[indx_completing_server]

            #record departure time
            D[indx_leaving_customer] = now
            numCompletedService += 1

            if numCompletedService >= numCustomers:
                break

            # check if there are people waiting
            if len(queue) > 0:
                customer_entering_service = queue[0]
                customers_in_service[indx_completing_server] = customer_entering_service
                server_completes_at[indx_completing_server] = now + np.random.exponential(1/mu)
                S[customer_entering_service] = now

                #remove the customer from the queue
                queue = queue[1:]
            else: # no people waiting
                customers_in_service[indx_completing_server] = -1
                server_completes_at[indx_completing_server] = np.inf

    return A, S, D


In [12]:
sim_one_path(1, .8, 2, 30)

IndexError: index 30 is out of bounds for axis 0 with size 30

#### For you:  Write a function that takes in the arrays A, S, D and computes 
 - The average waiting time across customers
 - The average length of the queue (averaged over time)

### (For you): Compute the expected average waiting time across the first 100 arrivals using a simulation with 1000 path

### More interesting service times
Suppose you believe that 10\% of customers have very difficult requests that take a long time for service, say they take $1/\mu_2 > 1/\mu_1$ time.  How would you alter your simulation?